### Project 3: Scikitlearn Logistic Regression
#### Montgomery College DATA201 Spring 2019  [Steve Dutky](sdutky@gmail.com)
The third project is on Logistic Regression
 The target/label variable is discrete/qualitative and could be binary or multi-class. Choose a suitable dataset. The main goal of this project is to use the Scikitlearn Logisitic Regression, split the data into training and testing data, check the accuracy (confusion matrix / AuC).

#### python/jupyter initialization:

In [523]:
## import packages:
import random # library of function for generating random numbers and samples
import numpy as np # for vector operations
import pandas as pd # for Data Frame methods
import matplotlib.pyplot as plt # Two dimensional plotting library
import seaborn as sns
from scipy import stats #  library of statistical functions
import sklearn #

#
# Set seed for generating psuedo random numbers
np.random.seed(12345678)
#
# Set up graphics
plt.style.use('classic')
%matplotlib inline

In [524]:
#murder=pd.read_csv("/home/dutky/Downloads/data-homicides-master/homicide-data.csv",engine='python')
murder=pd.read_csv("https://raw.githubusercontent.com/sdutky/mcData201/master/project3sources/homicide-data.csv",encoding='latin1')
murder.info()
murder.dropna(inplace=True)
murder.index=[i for i in range(murder.shape[0])]
murder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52179 entries, 0 to 52178
Data columns (total 12 columns):
uid              52179 non-null object
reported_date    52179 non-null int64
victim_last      52178 non-null object
victim_first     52179 non-null object
victim_race      52179 non-null object
victim_age       52179 non-null object
victim_sex       52179 non-null object
city             52179 non-null object
state            52179 non-null object
lat              52119 non-null float64
lon              52119 non-null float64
disposition      52179 non-null object
dtypes: float64(2), int64(1), object(9)
memory usage: 4.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 52118 entries, 0 to 52117
Data columns (total 12 columns):
uid              52118 non-null object
reported_date    52118 non-null int64
victim_last      52118 non-null object
victim_first     52118 non-null object
victim_race      52118 non-null object
victim_age       52118 non-null object
victim_sex       5

In [525]:
print(murder.describe().T)

                 count          mean           std           min  \
reported_date  52118.0  2.013091e+07  1.124076e+06  2.007010e+07   
lat            52118.0  3.702673e+01  4.348670e+00  2.572521e+01   
lon            52118.0 -9.147126e+01  1.374646e+01 -1.225078e+02   

                        25%           50%           75%           max  
reported_date  2.010032e+07  2.012122e+07  2.015091e+07  2.015111e+08  
lat            3.376518e+01  3.852491e+01  4.002763e+01  4.505119e+01  
lon           -9.599721e+01 -8.771029e+01 -8.175569e+01 -7.101152e+01  


In [526]:
murder.columns=['uid', 'date', 'victim_last', 'victim_first', 'race',
       'age', 'sex', 'city', 'state', 'lat', 'lon',
       'disposition']
murder.columns

Index(['uid', 'date', 'victim_last', 'victim_first', 'race', 'age', 'sex',
       'city', 'state', 'lat', 'lon', 'disposition'],
      dtype='object')

In [527]:
print(murder['disposition'].value_counts(),murder['race'].value_counts(),murder['sex'].value_counts())

Closed by arrest         25647
Open/No arrest           23556
Closed without arrest     2915
Name: disposition, dtype: int64 Black       33332
Hispanic     6888
White        6319
Unknown      4198
Other         698
Asian         683
Name: race, dtype: int64 Male       40696
Female      7192
Unknown     4230
Name: sex, dtype: int64


In [528]:
df=murder[['age','city','state','sex']].copy()

df['noArrest']=[int(murder.disposition[i] != 'Closed by arrest') for i in range(murder.shape[0])]

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 52118 entries, 0 to 52117
Data columns (total 5 columns):
age         52118 non-null object
city        52118 non-null object
state       52118 non-null object
sex         52118 non-null object
noArrest    52118 non-null int64
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


murder.loc[33547,'date']=20151118
murder.loc[33550,'date']=20151105
print(murder.loc[33547,'date'],murder.loc[33550,'date'])

In [529]:

a=['%d' % murder.date[i] for i in range(murder.shape[0]) ]
a[33547]='20151118'
a[33550]='20151105'


import time
# use try-catch to identify invalid dates
try: 
    for i in range(murder.shape[0]):
        [ time.strftime("%A", time.strptime(a[i],"%Y%m%d")) ]
        [ time.strftime("%b", time.strptime(a[i], "%Y%m%d")) ]
        
except ValueError as e:
    # e = sys.exc_info()[0]
    print( 'exception ', e.args)
    print(i,a[i])
    
df['day']=[ time.strftime("%A", time.strptime(a[i], "%Y%m%d")) for i in range(murder.shape[0]) ]
df['month']=[ time.strftime("%b", time.strptime(a[i], "%Y%m%d")) for i in range(murder.shape[0]) ]


In [530]:
# There are approx 69 miles per degree of longitude or latitude.
# We're going to recenter the geolocations to the nearest 2.5 mile circle:
lon=['%.4f' % (int(murder.lon[i]*1000*2.5/69)/1000*69/2.5) for i in range(murder.shape[0])]
lat=['%.4f' % (int(murder.lat[i]*1000*2.5/69)/1000*69/2.5) for i in range(murder.shape[0])]

df['zone']=[ lon[i]+'/'+lat[i] for i in range(murder.shape[0])]


In [531]:
df['race']=murder.race

In [532]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['zone','day','month','age'])
dfCoded = encoder.fit_transform(df)
dfCoded.drop(columns=['zone_0','day_0','month_0','age_0'],inplace=True)
dfCoded=pd.get_dummies(dfCoded,columns=['race','sex'])
print(dfCoded.describe().T)

                 count      mean       std  min  25%  50%  75%  max
zone_1         52118.0  0.325511  0.468570  0.0  0.0  0.0  1.0  1.0
zone_2         52118.0  0.298112  0.457433  0.0  0.0  0.0  1.0  1.0
zone_3         52118.0  0.413485  0.492463  0.0  0.0  0.0  1.0  1.0
zone_4         52118.0  0.533347  0.498891  0.0  0.0  1.0  1.0  1.0
zone_5         52118.0  0.518170  0.499675  0.0  0.0  1.0  1.0  1.0
zone_6         52118.0  0.541982  0.498239  0.0  0.0  1.0  1.0  1.0
zone_7         52118.0  0.536744  0.498653  0.0  0.0  1.0  1.0  1.0
zone_8         52118.0  0.513796  0.499814  0.0  0.0  1.0  1.0  1.0
zone_9         52118.0  0.502936  0.499996  0.0  0.0  1.0  1.0  1.0
zone_10        52118.0  0.511033  0.499883  0.0  0.0  1.0  1.0  1.0
zone_11        52118.0  0.494263  0.499972  0.0  0.0  0.0  1.0  1.0
zone_12        52118.0  0.511148  0.499881  0.0  0.0  1.0  1.0  1.0
day_1          52118.0  0.570283  0.495040  0.0  0.0  1.0  1.0  1.0
day_2          52118.0  0.605933  0.488654  0.0 

In [533]:
from sklearn.model_selection import train_test_split

predictorColummns= [a for a in dfCoded.columns if a!='city' and a!='state' and a!='noArrest' ]
predictors=dfCoded[predictorColummns].copy()
target=dfCoded[["noArrest"]].copy()
Xtrain, Xtest, yTrain, yTest = train_test_split( predictors, target, test_size=0.25, random_state=9)

In [534]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(solver='lbfgs')
logistic.fit(Xtrain,np.ravel(yTrain))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [535]:
#print(logistic.score(Xtest, yTest.values.ravel() )
print(logistic.score(Xtest, np.ravel(yTest) ))
print(logistic.score(Xtrain, np.ravel(yTrain) ))

0.5706062931696086
0.577031313958248


In [536]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(yTest, logistic.predict(Xtest)))

[[3250 3166]
 [2429 4185]]


In [537]:
print(pd.DataFrame({'features':Xtrain.columns,'coefficients':np.ravel(logistic.coef_)}))


         features  coefficients
0          zone_1     -0.305203
1          zone_2     -0.390979
2          zone_3     -0.164376
3          zone_4      0.085975
4          zone_5     -0.016073
5          zone_6      0.097760
6          zone_7      0.083910
7          zone_8      0.025382
8          zone_9      0.005604
9         zone_10     -0.002064
10        zone_11      0.005788
11        zone_12     -0.016146
12          day_1      0.037463
13          day_2      0.019468
14          day_3      0.010651
15        month_1      0.234356
16        month_2      0.070378
17        month_3      0.050565
18        month_4      0.035552
19          age_1     -0.206915
20          age_2      0.043600
21          age_3      0.008344
22          age_4      0.041455
23          age_5      0.028607
24          age_6     -0.022130
25          age_7      0.006530
26     race_Asian     -0.259580
27     race_Black      0.115007
28  race_Hispanic      0.131051
29     race_Other      0.009307
30   rac

In [538]:
cols={ 'zone':['zone_1', 'zone_2', 'zone_3', 'zone_4', 'zone_5', 'zone_6', 'zone_7', 'zone_8', 'zone_9', 'zone_10', 'zone_11', 'zone_12'], 'day':['day_1', 'day_2', 'day_3'],'month':[ 'month_1', 'month_2', 'month_3', 'month_4'], 'age':['age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7'], 'sex':['sex_Male','sex_Female','sex_Unknown'], 'race':['race_Asian', 'race_Black', 'race_Hispanic', 'race_Other', 'race_Unknown', 'race_White'] }
keys=list(cols.keys())
keys

['zone', 'day', 'month', 'age', 'sex', 'race']

In [539]:
import itertools

featureSet=list()
formulas=list()
for count in range(1, len(keys)+1):
    for combo in itertools.combinations(keys,count):
        a=list(combo)
        b=list()
        c='noArrest~'
        for i in range(len(a)):
            b+=cols[a[i]]
            c='%s%s+' % (c,a[i])
        featureSet.append(b)
        formulas.append(c[:-1])

best=pd.DataFrame({'formula':formulas,'scoreTrain':0,'scoreTest':0,'set':list(range(len(featureSet)))})
        

In [540]:
scoreTrain=list()
scoreTest=list()

for i in range(best.shape[0]):
    
    logistic.fit(Xtrain[featureSet[i]],np.ravel(yTrain))
    scoreTest+=[logistic.score(Xtest[featureSet[i]], np.ravel(yTest) )]
    scoreTrain+=[logistic.score(Xtrain[featureSet[i]], np.ravel(yTrain) )]
    
best.scoreTest=scoreTest
best.scoreTrain=scoreTrain
    

In [541]:
best.sort_values(by='scoreTrain',ascending=False)

,formula,scoreTrain,scoreTest,set
58,noArrest~zone+day+month+sex+race,0.577364,0.569225,58
60,noArrest~zone+month+age+sex+race,0.577185,0.572371,60
62,noArrest~zone+day+month+age+sex+race,0.577031,0.570606,62
49,noArrest~zone+month+sex+race,0.576622,0.568074,49
59,noArrest~zone+day+age+sex+race,0.572170,0.566078,59
46,noArrest~zone+day+sex+race,0.571633,0.564927,46
50,noArrest~zone+age+sex+race,0.571352,0.567613,50
30,noArrest~zone+sex+race,0.569766,0.563622,30
48,noArrest~zone+month+age+race,0.569433,0.562855,48
61,noArrest~day+month+age+sex+race,0.569407,0.566769,61
